# 1. Azure 솔루션을 활용한 Naive RAG 구현

- Disclaimer: 본 코드는 Azure의 보안 best practice를 고려하여 작성한 코드가 아닙니다. Azure 솔루션으로 RAG 구축 과정을 알아보는 것이 학습 목표이므로 최대한 코드를 간결하게 유지하기 위해 Azure의 Identity 관리 솔루션 등을 사용하지 않았습니다. 

- 참고 문헌:
  - [Build a RAG solution in Azure AI Search](https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Tutorial-RAG/Tutorial-rag.ipynb)
  - [mslearn-knowledge-mining](https://github.com/MicrosoftLearning/mslearn-knowledge-mining)

## 1.1 Indexing 구현하기

vector search 가능하게끔

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes import SearchIndexerClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex,
    SearchIndexerDataContainer,
    SearchIndexerDataSourceConnection
)

load_dotenv()
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_key = os.getenv("AZURE_OPENAI_API_KEY")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION")

In [ ]:
search_credential = AzureKeyCredential(search_key)

### 1.1.1 Index 정의하기

DB에 테이블 만들 때 각 컬럼별 속성 정의하듯이, 그런 느낌

In [ ]:
# Create a search index
index_name = "aoai-dev-day-index"
client = SearchIndexClient(endpoint=search_endpoint, credential=search_credential)
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),  
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),  
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile")
]

In [ ]:
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer_name="myOpenAI",  
        )
    ],  
    vectorizers=[  
        AzureOpenAIVectorizer(  
            vectorizer_name="myOpenAI",  
            kind="azureOpenAI",  
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=aoai_endpoint,
                api_key=aoai_key,
                deployment_name=embedding_deployment,
                model_name="text-embedding-3-large"
            ),
        ),  
    ], 
)  

In [ ]:
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  
result = client.create_or_update_index(index)
print(f"{result.name} created")

### 1.1.2 데이터 소스 만들기

In [ ]:
# Create a data source 
indexer_client = SearchIndexerClient(endpoint=search_endpoint, credential=search_credential)
container = SearchIndexerDataContainer(name="contoso")
data_source_connection = SearchIndexerDataSourceConnection(
    name="aoai-dev-day-datasource",
    type="azureblob",
    connection_string=storage_connection_string,
    container=container
)
data_source = indexer_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

### 1.1.3 Skillset 만들기

- 어떤 Skill을 써서 Indexing을 진행하고, 어느 Index 필드에 결과물을 맵핑할 지 

In [ ]:
from azure.search.documents.indexes.models import (
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerSkillset
)

# Create a skillset  
skillset_name = "aoai-dev-day-skillset"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=300,  
    page_overlap_length=0,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/content"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ],  
)  
  
embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_url=aoai_endpoint,
    api_key=aoai_key,
    deployment_name=embedding_deployment,  
    model_name="text-embedding-3-large",
    dimensions=1536,
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="text_vector")  
    ],  
)

index_projections = SearchIndexerIndexProjection(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=index_name,
            parent_key_field_name="parent_id",  
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),  
                InputFieldMappingEntry(name="text_vector", source="/document/pages/*/text_vector"),
                InputFieldMappingEntry(name="title", source="/document/metadata_storage_name"),  
            ],  
        ),  
    ],  
    parameters=SearchIndexerIndexProjectionsParameters(  
        projection_mode=IndexProjectionMode.SKIP_INDEXING_PARENT_DOCUMENTS  
    ),  
) 

In [ ]:
skills = [split_skill, embedding_skill]

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents and generating embeddings",  
    skills=skills,  
    index_projection=index_projections
)
  
indexer_client.create_or_update_skillset(skillset)  
print(f"{skillset.name} created")

### 1.1.4 Indexer 만들기

- data source, indexer, index 연결하는 구성 요소

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndexer
)

# Create an indexer  
indexer_name = "aoai-dev-day-indexer" 

indexer_parameters = None

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to index documents and generate embeddings",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name,
    parameters=indexer_parameters
)  

# Create and run the indexer  
indexer_result = indexer_client.create_or_update_indexer(indexer)  

print(f' {indexer_name} is created and running. Give the indexer a few minutes before running a query.')  

## 1.2 Retrieval 구현하기 (Vector Search를 활용하여)

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

# Vector Search using text-to-vector conversion of the query string
query = "임플란트 했는데 사내 실비 보험으로 청구 가능한가요?"  

search_client = SearchClient(endpoint=search_endpoint, credential=search_credential, index_name=index_name)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="text_vector")
  
results = search_client.search(  
    # search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=4
)  
  
for result in results:  
    print(f"Score: {result['@search.score']}")
    print(f"Chunk: {result['chunk']}")

## 1.3 Augmentation 단계 구현하기

In [ ]:
# Provide instructions to the model
PROMPT="""
너는 인사 지원 에이전트야. 친절하게 이모티콘을 많이써서 답변하고, 잘 모르는 내용이 있다면 HR팀에 문의해야 한다고 답변할 것.
Query: {query}
Sources:\n{sources}
"""

In [ ]:
results = search_client.search(  
    # search_text=query,  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=4
)  

sources_formatted = "====\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in results])

In [ ]:
sources_formatted

In [ ]:
AUGMENTED_QUERY = PROMPT.format(query=query, sources=sources_formatted)

## 1.4 Generation 단계 구현하기

### 1.4.1 RAG 안한 것

In [ ]:
# Import libraries
from openai import AzureOpenAI

openai_client = AzureOpenAI(
     api_version="2024-12-01-preview",
     azure_endpoint=aoai_endpoint,
     api_key=aoai_key
 )

deployment_name = chat_deployment

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query
        }
    ],
    model=deployment_name
)

print(response.choices[0].message.content)

### 1.4.2 RAG 적용한 것

In [ ]:
# Import libraries
from openai import AzureOpenAI

# Set up the Azure OpenAI client
openai_client = AzureOpenAI(
     api_version="2024-12-01-preview",
     azure_endpoint=aoai_endpoint,
     api_key=aoai_key
 )

deployment_name = chat_deployment

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": AUGMENTED_QUERY
        }
    ],
    model=deployment_name
)

print(response.choices[0].message.content)

## 1.5 Naive RAG 전체 흐름 테스트 해보기

In [ ]:
# Retrieval
query = "임플란트 했는데 사내 실비 보험으로 청구 가능한가요?"  

search_client = SearchClient(endpoint=search_endpoint, credential=search_credential, index_name=index_name)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="text_vector")
  
results = search_client.search(  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=4
)

#Augmented
PROMPT="""
너는 인사 지원 에이전트야. 친절하게 이모티콘을 많이써서 답변하고, 잘 모르는 내용이 있다면 HR팀에 문의해야 한다고 답변할 것.
Query: {query}
Sources:\n{sources}
"""

sources_formatted = "====\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in results])

AUGMENTED_QUERY = PROMPT.format(query=query, sources=sources_formatted)

#Generation
openai_client = AzureOpenAI(
     api_version="2024-12-01-preview",
     azure_endpoint=aoai_endpoint,
     api_key=aoai_key
 )

deployment_name = chat_deployment

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": AUGMENTED_QUERY
        }
    ],
    model=deployment_name
)

print(response.choices[0].message.content)


## 1.6 Naive RAG 구조가 한계를 보이는 시나리오 확인하기

In [ ]:
# Retrieval
query = "치과에서 임플란트랑 충치 치료를 최근에 했는데 사내 실비 보험으로 청구 가능한가요? 그리고 수술 관련 때문에 입원을 해야 할 것 같은데 병가를 사용할 수 있나요?"

search_client = SearchClient(endpoint=search_endpoint, credential=search_credential, index_name=index_name)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="text_vector")
  
results = search_client.search(  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=4
)

#Augmented
PROMPT="""
너는 인사 지원 에이전트야. 친절하게 이모티콘을 많이써서 답변하고, 참고해야 하는 자료 중에 관련 내용이 없다면 모르겠다고 답변할것. 그리고 관련 문의는 HR팀에게 해야 한다고 답변할 것.
Query: {query}
Sources:\n{sources}
"""

sources_formatted = "====\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in results])

AUGMENTED_QUERY = PROMPT.format(query=query, sources=sources_formatted)

#Generation
openai_client = AzureOpenAI(
     api_version="2024-12-01-preview",
     azure_endpoint=aoai_endpoint,
     api_key=aoai_key
 )

deployment_name = chat_deployment

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": AUGMENTED_QUERY
        }
    ],
    model=deployment_name,
    temperature=0
)

print(response.choices[0].message.content)


In [ ]:
# Retrieval
query = "치과에서 임플란트랑 충치 치료를 최근에 했는데 사내 실비 보험으로 청구 가능한가요? 그리고 수술 관련 때문에 입원을 해야 할 것 같은데 병가를 사용할 수 있나요?"

search_client = SearchClient(endpoint=search_endpoint, credential=search_credential, index_name=index_name)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="text_vector")
  
results = search_client.search(  
    vector_queries= [vector_query],
    select=["title", "chunk"],
    top=4
)

for result in results:  
    print(f"Score: {result['@search.score']}")
    print(f"Chunk: {result['title']}")